In [ ]:
# Verification script to check basic PaSD functionality using the low-ITF hardware or simulation

In [ ]:
import json
import time
import numpy
import random

In [ ]:
# Run this to connect to a simulation in minikube:
# ------------------------------------------------
pasd=tango.DeviceProxy("low-mccs/pasdbus/ci-1")
fndh=tango.DeviceProxy("low-mccs/fndh/ci-1")
fncc=tango.DeviceProxy("low-mccs/fncc/ci-1")
sb1=tango.DeviceProxy("low-mccs/smartbox/ci-1-sb01")
sb2=tango.DeviceProxy("low-mccs/smartbox/ci-1-sb02")

In [ ]:
# Run this to connect to the low-ITF hardware:
# --------------------------------------------
pasd=tango.DeviceProxy("low-mccs/pasdbus/s8-1")
fndh=tango.DeviceProxy("low-mccs/fndh/s8-1")
fncc=tango.DeviceProxy("low-mccs/fncc/s8-1")
sb1=tango.DeviceProxy("low-mccs/smartbox/s8-1-sb01")
sb2=tango.DeviceProxy("low-mccs/smartbox/s8-1-sb02")

In [ ]:
# Setup: start communicating with the FNDH
pasd.adminMode=0
time.sleep(5)
pasd.initializeFndh()
fndh.adminMode=0
time.sleep(10)

In [ ]:
# FNDH attributes should now be populated
assert fndh.state() == tango._tango.DevState.ON
print(f"Uptime: {fndh.uptime}")
print(f"ModbusRegisterMapRevisionNumber: {fndh.ModbusRegisterMapRevisionNumber}")
print(f"PcbRevisionNumber: {fndh.PcbRevisionNumber}")
print(f"CpuId: {fndh.CpuId}")
print(f"ChipId: {fndh.ChipId}")
print(f"FirmwareVersion: {fndh.FirmwareVersion}")
print(f"SysAddress: {fndh.SysAddress}")
print(f"Psu48vVoltage1: {fndh.Psu48vVoltage1}")
print(f"Psu48vVoltage2: {fndh.Psu48vVoltage2}")
print(f"Psu48vCurrent: {fndh.Psu48vCurrent}")
print(f"Psu48vTemperature1: {fndh.Psu48vTemperature1}")
print(f"Psu48vTemperature2: {fndh.Psu48vTemperature2}")
print(f"PanelTemperature: {fndh.PanelTemperature}")
print(f"FncbTemperature: {fndh.FncbTemperature}")
print(f"FncbHumidity: {fndh.FncbHumidity}")
print(f"PasdStatus: {fndh.PasdStatus}")
print(f"LedPattern: {fndh.LedPattern}")
print(f"CommsGatewayTemperature: {fndh.CommsGatewayTemperature}")
print(f"PowerModuleTemperature: {fndh.PowerModuleTemperature}")
print(f"OutsideTemperature: {fndh.OutsideTemperature}")
print(f"InternalAmbientTemperature: {fndh.InternalAmbientTemperature}")
print(f"PortsPowerSensed: {fndh.PortsPowerSensed}")
print(f"WarningFlags: {fndh.WarningFlags}")
print(f"AlarmFlags: {fndh.AlarmFlags}")
print(f"Psu48vVoltage1Thresholds: {fndh.Psu48vVoltage1Thresholds}")
print(f"Psu48vVoltage2Thresholds: {fndh.Psu48vVoltage2Thresholds}")
print(f"Psu48vCurrentThresholds: {fndh.Psu48vCurrentThresholds}")
print(f"Psu48vTemperature1Thresholds: {fndh.Psu48vTemperature1Thresholds}")
print(f"Psu48vTemperature2Thresholds: {fndh.Psu48vTemperature2Thresholds}")
print(f"PanelTemperatureThresholds: {fndh.PanelTemperatureThresholds}")
print(f"FncbTemperatureThresholds: {fndh.FncbTemperatureThresholds}")
print(f"FncbHumidityThresholds: {fndh.FncbHumidityThresholds}")
print(f"CommsGatewayTemperatureThresholds: {fndh.CommsGatewayTemperatureThresholds}")
print(f"PowerModuleTemperatureThresholds: {fndh.PowerModuleTemperatureThresholds}")
print(f"OutsideTemperatureThresholds: {fndh.OutsideTemperatureThresholds}")
print(f"InternalAmbientTemperatureThresholds: {fndh.InternalAmbientTemperatureThresholds}")

In [ ]:
# Save initial state so that we can restore after the tests
fndh_initial_state = fndh.portsPowerSensed
print (f"Initial FNDH port state: {fndh_initial_state}")

In [ ]:
# First ensure all smartboxes are turned off so that we can test turning them on
port_powers_off = json.dumps({"port_powers": 
    [False, False, False, False, False, False, False, False, False, False, False, False, False, False,
     False, False, False, False, False, False, False, False, False, False, False, False, False, False], "stay_on_when_offline": False}) 
pasd.setFndhPortPowers(port_powers_off)
time.sleep(5)
assert (fndh.portsPowerSensed == numpy.array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])).all()

In [ ]:
# Turn on smartboxes 1 and 2
port_powers_on = json.dumps({"port_powers": 
    [True, True, False, True, False, True, False, False, False, False, False, False, False, False,
     False, False, False, False, False, False, False, False, False, False, False, False, False, False], "stay_on_when_offline": False}) 
pasd.setFndhPortPowers(port_powers_on)
time.sleep(15)
assert (fndh.portsPowerSensed == numpy.array([True, True, False, True, False, True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])).all()

In [ ]:
# Start communicating with the smartboxes
sb1.adminMode=0
sb2.adminMode=0
pasd.initializeSmartbox(1)
pasd.initializeSmartbox(2)
sbs = {1: sb1, 2: sb2}
time.sleep(10)
for i in range(1,3):
    print(f"Smartbox {i} Uptime: {sbs[i].uptime}")
    print(f"Smartbox {i} ModbusRegisterMapRevisionNumber: {sbs[i].ModbusRegisterMapRevisionNumber}")
    print(f"Smartbox {i} PcbRevisionNumber: {sbs[i].PcbRevisionNumber}")
    print(f"Smartbox {i} CpuId: {sbs[i].CpuId}")
    print(f"Smartbox {i} ChipId: {sbs[i].ChipId}")
    print(f"Smartbox {i} FirmwareVersion: {sbs[i].FirmwareVersion}")
    print(f"Smartbox {i} SysAddress: {sbs[i].SysAddress}")
    print(f"Smartbox {i} InputVoltage: {sbs[i].InputVoltage}")
    print(f"Smartbox {i} PowerSupplyOutputVoltage: {sbs[i].PowerSupplyOutputVoltage}")
    print(f"Smartbox {i} PcbTemperature: {sbs[i].PcbTemperature}")
    print(f"Smartbox {i} FemAmbientTemperature: {sbs[i].FemAmbientTemperature}")
    print(f"Smartbox {i} PasdStatus: {sbs[i].PasdStatus}")
    print(f"Smartbox {i} LedPattern: {sbs[i].LedPattern}")
    print(f"Smartbox {i} FemCaseTemperature1: {sbs[i].FemCaseTemperature1}")
    print(f"Smartbox {i} FemCaseTemperature2: {sbs[i].FemCaseTemperature2}")
    print(f"Smartbox {i} FemHeatsinkTemperature1: {sbs[i].FemHeatsinkTemperature1}")
    print(f"Smartbox {i} FemHeatsinkTemperature2: {sbs[i].FemHeatsinkTemperature2}")
    print(f"Smartbox {i} PowerSupplyTemperature: {sbs[i].PowerSupplyTemperature}")
    print(f"Smartbox {i} PortBreakersTripped: {sbs[i].PortBreakersTripped}")
    print(f"Smartbox {i} PortsPowerSensed: {sbs[i].PortsPowerSensed}")
    print(f"Smartbox {i} WarningFlags: {sbs[i].WarningFlags}")
    print(f"Smartbox {i} AlarmFlags: {sbs[i].AlarmFlags}")
    print(f"Smartbox {i} FemCurrentTripThresholds: {sbs[i].FemCurrentTripThresholds}")

In [ ]:
# Save initial state and then ensure all FEMs are all turned off to start with
sb1_initial_state = sb1.portsPowerSensed
print(f"Smartbox 1 initial port state: {sb1_initial_state}")
sb1_port_powers_off = json.dumps(
   {"smartbox_number": 1, "port_powers": [False, False, False, False, False, False, False, False, False, False, False, False, ], "stay_on_when_offline": True})
pasd.setSmartboxPortPowers(sb1_port_powers_off)
time.sleep(5)
assert (sb1.portsPowerSensed == numpy.array([False, False, False, False, False, False, False, False, False,
       False, False, False])).all()

In [ ]:
# Turn on ports 1 and 2
sb1_port_powers_on = json.dumps(
   {"smartbox_number": 1, "port_powers": [True, True, False, True, False, False, False, False, False, False, False, False, ], "stay_on_when_offline": True})
pasd.setSmartboxPortPowers(sb1_port_powers_on)
time.sleep(10)
assert (sb1.portsPowerSensed == numpy.array([True, True, False, True, False, False, False, False, False, False, False, False])).all()

In [ ]:
# Test setting alarm thresholds
initial_humidity_thresholds = fndh.FncbHumidityThresholds
current_humidity = fndh.fncbHumidity
alarm_value = random.randint(0, current_humidity-2)
fndh.FncbHumidityThresholds = [alarm_value, 3, 2, 1]
time.sleep(10)
assert (fndh.FncbHumidityThresholds == numpy.array([alarm_value, 3, 2, 1])).all()

In [ ]:
time.sleep(30)
assert fndh.alarmFlags == 'SYS_HUMIDITY'

In [ ]:
# Set the thresholds back and reset the alarm
fndh.FncbHumidityThresholds = initial_humidity_thresholds
time.sleep(5)
assert (fndh.FncbHumidityThresholds == numpy.array([initial_humidity_thresholds])).all()

In [ ]:
pasd.initializeFndh()
time.sleep(5)
pasd.ResetFndhAlarms()
time.sleep(30)
assert fndh.alarmFlags == 'NONE'

In [ ]:
# Restore initial state and terminate connection
sb1_initial_port_powers = json.dumps({"smartbox_number": 1, "port_powers": sb1_initial_state.tolist(), "stay_on_when_offline": False})
pasd.setSmartboxPortPowers(sb1_initial_port_powers)
time.sleep(5)
assert (sb1.portsPowerSensed == sb1_initial_state).all()

In [ ]:
fndh_initial_port_powers = json.dumps({"port_powers": fndh_initial_state.tolist(), "stay_on_when_offline": False})
pasd.setFndhPortPowers(fndh_initial_port_powers)
time.sleep(5)
assert (fndh.portsPowerSensed == fndh_initial_state).all()

In [ ]:
pasd.adminMode=1